In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def sphere_to_cartesian(r, gamma, theta):
    '''Convert spherical coordinates to cartesian
       r - sphere radius
       gamma - polar angle
       theta - azimuth angle
       return - cartesian coordinates
    '''
    return (r * np.sin(gamma) * np.sin(theta),
            r * np.sin(gamma) * np.cos(theta),
            r * np.cos(gamma))

def s_arc(sr, c_gamma, c_theta, r_delta, start, end, n=32):
    '''Get arc points plotted on a sphere's surface
       sr - sphere radius
       c_gamma - polar angle of the circle's center
       c_theta - azimuth angle of the circle's center
       r_delta - angle between OC and OP, where O is sphere's center,
                 C is the circle's center, P is any point on the circle
       start - arc start angle
       end - arc end angle
       n - number of points
       return - circle points
    '''
    t = np.expand_dims(np.linspace(start, end, n), axis=1)
    a = sphere_to_cartesian(1.0, c_gamma + r_delta, c_theta)
    k = sphere_to_cartesian(1.0, c_gamma, c_theta)
    c = np.cos(t) * a + np.sin(t) * np.cross(k, a) + np.dot(k, a) * (1.0 - np.cos(t)) * k
    c = c * sr
    return (dim.squeeze() for dim in np.hsplit(c, 3))

def s_inv(gamma0, gamma):
    phi = np.arccos(np.tan(gamma0) / np.tan(gamma))    
    return np.arccos(np.cos(gamma) / np.cos(gamma0)) / np.sin(gamma0) - phi


def circle3d_by3points(a, b, c):
    u = b - a
    w = np.cross(c - a, u)
    u = u / np.linalg.norm(u)
    w = w / np.linalg.norm(w)
    v = np.cross(w, u)
    
    bx = np.dot(b - a, u)
    cx, cy = np.dot(c - a, u), np.dot(c - a, v)
    
    h = ((cx - bx / 2.0) ** 2 + cy ** 2 - (bx / 2.0) ** 2) / (2.0 * cy)
    cc = a + u * (bx / 2.0) + v * h
    r = np.linalg.norm(a - cc)

    return r, cc


def rotate_points_around_axis(p, u, a):
    ux, uy, uz = u
    sina, cosa = np.sin(a), np.cos(a)
    r_mat = np.array((
        (cosa + (1.0 - cosa) * ux ** 2,
         ux * uy * (1.0 - cosa) - uz * sina,
         ux * uz * (1.0 - cosa) + uy * sina),
        (uy * ux * (1.0 - cosa) + uz * sina,
         cosa + (1.0 - cosa) * uy ** 2,
         uy * uz * (1.0 - cosa) - ux * sina),
        (uz * ux * (1.0 - cosa) - uy * sina,
         uz * uy * (1.0 - cosa) + ux * sina,
         cosa + (1.0 - cosa) * uz ** 2),
    ))

    return p @ r_mat


def angle_between(o, a, b):
    p = a - o
    q = b - o
    return np.arccos(np.dot(p, q) / (np.linalg.norm(p) * np.linalg.norm(q)))


In [ ]:
m = 1.0 # module
z = 120  # teeth number
a = np.radians(20.0) # pressure angle
fw = 3.0 # gear face width
ka = 1.0 # addendum coefficient
kd = 1.25 # dedendum coefficient

gamma_p = np.radians(45.0) # pitch cone angle

rp = m * z / 2.0 # base/pitch circle radius
gs_r = rp / np.sin(gamma_p) # great sphere radius/pitch cone flank length

gamma_b = np.arcsin(np.cos(a) * np.sin(gamma_p))
gamma_f = gamma_p + np.arctan(ka * m / gs_r)
gamma_r = gamma_p - np.arctan(kd * m / gs_r)

phi_r = s_inv(gamma_b, gamma_p);
mirrpoint = np.pi / z + 2.0 * phi_r

tau = np.pi * 2.0 / z



In [234]:
fig = plt.figure()
ax = plt.axes(projection='3d')

# Great sphere
u = np.linspace(0.0, 2.0 * np.pi, 24)
v = np.linspace(0.0, np.pi, 24)
gs_x = gs_r * np.outer(np.cos(u), np.sin(v))
gs_y = gs_r * np.outer(np.sin(u), np.sin(v))
gs_z = gs_r * np.outer(np.ones(np.size(u)), np.cos(v))
ax.plot_surface(gs_x, gs_y, gs_z, color='#aaa', alpha=0.1)

# Pitch circle
t = np.linspace(0.0, np.pi * 2.0, 64)
pc_x, pc_y, pc_z = sphere_to_cartesian(gs_r, gamma_p, t)
ax.plot3D(pc_x, pc_y, pc_z, linestyle='dashed', linewidth=0.5, alpha=0.8)

# Base circle
bc_x, bc_y, bc_z = sphere_to_cartesian(gs_r, gamma_b, t)
ax.plot3D(bc_x, bc_y, bc_z, linestyle='dashed', linewidth=0.5, alpha=0.8)

# Face circle
fc_x, fc_y, fc_z = sphere_to_cartesian(gs_r, gamma_f, t)
ax.plot3D(fc_x, fc_y, fc_z, linestyle='dashed', linewidth=0.5, alpha=0.8)

# Root circle
rc_x, rc_y, rc_z = sphere_to_cartesian(gs_r, gamma_r, t)
ax.plot3D(rc_x, rc_y, rc_z, linestyle='dashed', linewidth=0.5, alpha=0.8)

# ss_x, ss_y, ss_z = s_circle(gs_r, np.pi / 4.0, np.pi, 0.9)
# ax.plot3D(ss_x, ss_y, ss_z, linewidth=1.0)

# Tooth body
gamma = np.linspace(gamma_b, gamma_f, 16)
theta = s_inv(gamma_b, gamma)
tooth_lflank = np.dstack(sphere_to_cartesian(gs_r, gamma, theta)).squeeze()
theta_tip = np.linspace(theta[-1], mirrpoint - theta[-1], 16)
tooth_tip = np.dstack(sphere_to_cartesian(gs_r, np.full(16, gamma_f), theta_tip)).squeeze()
tooth_rflank = np.dstack(sphere_to_cartesian(gs_r, gamma[::-1], mirrpoint - theta[::-1])).squeeze()

tooth_p = np.vstack((tooth_lflank, tooth_tip, tooth_rflank))
tf_x, tf_y, tf_z = (ax.squeeze() for ax in np.hsplit(tooth_p, 3))
ax.plot3D(tf_x, tf_y, tf_z, linewidth=1.0)

# Tooth root
if gamma_r < gamma_b:
    p1 = tooth_rflank[-1]
    p2 = np.array(sphere_to_cartesian(gs_r, gamma_b, theta[0] + tau))
    p3 = np.array(sphere_to_cartesian(gs_r, gamma_r, (tau + mirrpoint) / 2.0))
    rr, rcc = circle3d_by3points(p1, p2, p3)
    rcc_gamma = np.arccos(np.dot(p3, rcc) / (np.linalg.norm(p3) * np.linalg.norm(rcc)))

    # px, py, pz = np.hsplit(np.vstack((rcc, p1, p2, p3)), 3)
    # ax.scatter(px, py, pz)

    p1p3 = angle_between(rcc, p1, p3)
    a_start = (np.pi - p1p3 * 2.0) / 2.0
    a_end = -a_start + np.pi
    rcx, rcy, rcz = s_arc(gs_r, gamma_r + rcc_gamma,
                          (tau + mirrpoint) / 2.0, rcc_gamma,
                          np.pi / 2.0 + a_start, np.pi / 2.0 + a_end)
    ax.plot3D(rcx, rcy, rcz, linewidth=1.0)
else:
    r_theta = np.linspace(mirrpoint - theta[0], theta[0] + tau, 16)
    rcx, rcy, rcz = sphere_to_cartesian(gs_r, np.full(16, gamma_b), r_theta)
    ax.plot3D(rcx, rcy, rcz, linewidth=1.0)

print(theta[0], mirrpoint - theta[0], tau)
    
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.7706678376228314 0.726775054319697 0.7971827308556025
0.0 0.06737662449410571 0.05235987755982988


In [ ]:
def circle3d_by3points(a, b, c):
    u = b - a
    w = np.cross(c - a, u)
    u = u / np.linalg.norm(u)
    w = w / np.linalg.norm(w)
    v = np.cross(w, u)
    
    bx = np.dot(b - a, u)
    cx, cy = np.dot(c - a, u), np.dot(c - a, v)
    
    h = ((cx - bx / 2.0) ** 2 + cy ** 2 - (bx / 2.0) ** 2) / (2.0 * cy)
    cc = a + u * (bx / 2.0) + v * h
    r = np.linalg.norm(a - cc)

    return r, cc
    

In [ ]:
r = 7.0
a = np.array((np.cos(0.0) * r, np.sin(0.0) * r, 0.0))
b = np.array((np.cos(np.pi / 4.0) * r, np.sin(np.pi / 4.0) * r, 0.0))
c = np.array((np.cos(np.pi / 2.0) * r, np.sin(np.pi / 2.0) * r, 0.0))

circle3d_by3points(a, b, c)


In [ ]:
def rotate_points_around_axis(p, u, a):
    ux, uy, uz = u
    sina, cosa = np.sin(a), np.cos(a)
    r_mat = np.array((
        (cosa + (1.0 - cosa) * ux ** 2,
         ux * uy * (1.0 - cosa) - uz * sina,
         ux * uz * (1.0 - cosa) + uy * sina),
        (uy * ux * (1.0 - cosa) + uz * sina,
         cosa + (1.0 - cosa) * uy ** 2,
         uy * uz * (1.0 - cosa) - ux * sina),
        (uz * ux * (1.0 - cosa) - uy * sina,
         uz * uy * (1.0 - cosa) + ux * sina,
         cosa + (1.0 - cosa) * uz ** 2),
    ))

    return p @ r_mat

p1 = np.array((7.0, 0.0, 13.0))
p2 = np.array((0.0, 42.0, 14.0))

pts = np.vstack((p1, p2))
print(pts)
rotate_points_around_axis(pts, (0.0, 0.0, 1.0), np.pi)